In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [7]:
# Column Schema

FINAL_COLUMNS = [
    "date", "sunrise", "sunset", "daylength", "daylength_difference",
    "at_start", "at_end", "nt_start", "nt_end", "ct_start", "ct_end",
    "sn_time", "sn_mil_km"
]

TIME_COLS = [
    "sunrise", "sunset",
    "at_start", "at_end",
    "nt_start", "nt_end",
    "ct_start", "ct_end",
    "sn_time",
]

DURATION_COLS = ["daylength", "daylength_difference"]

FLOAT_COLS = ["sn_mil_km"]

In [8]:
# Helpers

def parse_sheet_year_month(sheet_name):
    """Convert sheet name 'YYMM' → (year, month)."""
    yy = int(sheet_name[:2])
    mm = int(sheet_name[2:])
    year = 2000 + yy if yy < 50 else 1900 + yy
    return year, mm


def clean_time_string(value):
    """Extract the first HH:MM or HH:MM:SS from the string."""
    if not isinstance(value, str):
        return value

    m = re.search(r"\b\d{1,2}:\d{2}(?::\d{2})?\b", value)
    return m.group(0) if m else ""

In [9]:
# Load each sheet

def load_single_sheet(path, sheet_name):
    year, month = parse_sheet_year_month(sheet_name)

    df = pd.read_excel(path, sheet_name=sheet_name, header=1)

    # Remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains("Unnamed")]

    # First column is day
    day_col = df.columns[0]
    df[day_col] = pd.to_numeric(df[day_col], errors="coerce")

    # Drop blank days
    df = df.dropna(subset=[day_col])

    # Create full date
    df["Date"] = pd.to_datetime({
        "year": year,
        "month": month,
        "day": df[day_col].astype(int)
    })

    df = df.drop(columns=[day_col])

    # Clean times
    df = df.map(clean_time_string)

    # Move Date to front
    df = df[["Date"] + [c for c in df.columns if c != "Date"]]

    return df

In [10]:
# load all sheets

def load_all_sheets(path):
    xls = pd.ExcelFile(path)
    frames = [
        load_single_sheet(path, name)
        for name in xls.sheet_names
        if re.fullmatch(r"\d{4}", name)
    ]
    return pd.concat(frames, ignore_index=True)

In [ ]:
# convert to schema
def to_time(val):
    """Convert anything → datetime.time or None."""
    if isinstance(val, datetime.time):
        return val
    if isinstance(val, datetime.datetime):
        return val.time()
    if isinstance(val, float) and pd.isna(val):
        return None
    if isinstance(val, str) and val.strip() == "":
        return None
    try:
        t = pd.to_datetime(val, errors="coerce")
        if pd.isna(t):
            return None
        return t.time()
    except:
        return None
    


In [12]:
def to_timedelta(val):
    """Convert strings / time objects / datetime objects → timedelta."""
    if isinstance(val, datetime.timedelta):
        return val
    if isinstance(val, datetime.time):
        return datetime.timedelta(
            hours=val.hour, minutes=val.minute, seconds=val.second
        )
    if isinstance(val, datetime.datetime):
        val = val.time()
        return datetime.timedelta(
            hours=val.hour, minutes=val.minute, seconds=val.second
        )
    if isinstance(val, float) and pd.isna(val):
        return pd.NaT
    try:
        return pd.to_timedelta(val)
    except:
        return pd.NaT

In [13]:
# Main processing
df = load_all_sheets("data/Edinburgh-daytime.xlsx")

# Rename to final column names
df.columns = FINAL_COLUMNS[:len(df.columns)]
df = df[FINAL_COLUMNS]

# date → datetime64 (date only)
df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
df["date"] = pd.to_datetime(df["date"])  # convert back to datetime64

# Times → datetime.time
for col in TIME_COLS:
    df[col] = df[col].apply(to_time)

# Durations → timedelta
for col in DURATION_COLS:
    df[col] = df[col].apply(to_timedelta)

# Floats
for col in FLOAT_COLS:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype(float)

In [14]:
# output

output_path = "data/edinburgh-daytime-cleaned.csv"
df.to_csv(output_path, index=False)

print("Saved:", output_path)

Saved: data/edinburgh-daytime-cleaned.csv
